In [62]:
import nest_asyncio

nest_asyncio.apply()

In [63]:
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import BaseNode, Document, IndexNode, NodeWithScore

from readai.components.retrievers import BM25Retriever


### 数据预处理

#### 加载md文件

In [64]:
# 加载.env文件
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

# 获取项目根目录路径
project_root = Path(os.getenv("PROJECT_ROOT"))

In [65]:
test_data_path = project_root / "readai/tests/data"

In [66]:
book_name = "comunication_cleaned.md"
book_path = test_data_path / book_name
# 使用unstructured的markdownloader加载
loader = UnstructuredMarkdownLoader(book_path, mode="elements")
documents = loader.load()
# 查看节点数量
print(f"节点数量: {len(documents)}")
# 遍历documents,重构metadata，只保留category，filename这两个属性
for doc in documents:
    category = doc.metadata["category"]
    filename = doc.metadata["filename"]
    doc.metadata = {
        "category": category,
        "filename": filename,
    }

节点数量: 41


In [67]:
llamindex_docs = [
    Document(
        text=doc.page_content,
        metadata=doc.metadata,
        metadata_seperator="::",
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    for doc in documents
]

In [68]:
print(len(llamindex_docs))
print(llamindex_docs[0].text, "\n\n", llamindex_docs[0].metadata)

41
马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org. 版权所有翻印必究 北京市版权局著作权合同登记号：图字01-2016-2253号 非暴力

In [69]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import IndexNode

node_parser = SimpleNodeParser.from_defaults(
    chunk_size=1024,
    chunk_overlap=50,
    separator="\n\n",
    secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
)
hybrid_nodes = node_parser.get_nodes_from_documents(llamindex_docs, show_progress=True)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

需要用hybrid_nodes来构建测试集？1

In [199]:
# 通过pandas来检查hybrid_nodes的text长度分布情况
import pandas as pd

# 创建一个DataFrame来存储节点信息
nodes_df = pd.DataFrame(
    {
        "node_id": [node.node_id for node in hybrid_nodes],
        "text_length": [len(node.text) for node in hybrid_nodes],
    }
)

# 计算text长度分布情况
nodes_df["text_length"].describe()


count     170.000000
mean      714.541176
std       308.807788
min         2.000000
25%       782.750000
50%       835.000000
75%       862.750000
max      1778.000000
Name: text_length, dtype: float64

因为hybrid_nodes是所有small to big节点的源节点，并且也被添加到了hybrid_all_nodes中，所以可以用于构建测试集

In [197]:
print(hybrid_nodes[0].text)
print(len(hybrid_nodes))

马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org.
170


#### small2big方法

In [71]:
sub_chunk_sizes = [128, 512, 2048]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(
        chunk_size=c,
        chunk_overlap=50,
        separator="\n\n",
        secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
    )
    for c in sub_chunk_sizes
]

hybrid_all_nodes = []
for hybrid_node in hybrid_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([hybrid_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, hybrid_node.node_id) for sn in sub_nodes
        ]
        hybrid_all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(hybrid_node, hybrid_node.node_id)
    hybrid_all_nodes.append(original_node)

print(len(hybrid_all_nodes))

2753


In [196]:
for node in hybrid_all_nodes[:10]:
    print(node.node_id)

045b921e-231e-4a73-a426-b87f95c05aed
c25c4cd4-ccc8-41dd-a4ae-6e24e2049c25
9d12db15-bf0c-4245-834f-163a0c5c73eb
9c71d44b-14c5-4169-870e-405d8c1baba3
f3b5de80-9d4f-4444-8b1c-7c3c5e026e0b
a602414f-3d04-4277-9e23-50833eaa69b9
51149c86-2858-47c3-9058-26af4aeb57be
83c14538-4c21-4de5-974c-96c0622b66c1
ac192ea5-90c8-46b2-8166-7a4d1258e9a8
2d25be2b-86d1-4104-82ef-1c26df2e514d


In [73]:
hybrid_all_nodes_dict = {n.node_id: n for n in hybrid_all_nodes}

In [74]:
# 查看前5个
for node_id, node in list(hybrid_all_nodes_dict.items())[:5]:
    print(node_id, node.text)

045b921e-231e-4a73-a426-b87f95c05aed 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。
c25c4cd4-ccc8-41dd-a4ae-6e24e2049c25 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！
9d12db15-bf0c-4245-834f-163a0c5c73eb 言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。
9c71d44b-14c5-4169-870e-405d8c1baba3 这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦
f3b5de80-9d4f-4444-8b1c-7c3c5e026e0b 伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。


### 向量库加载

In [75]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.deepseek import DeepSeek

api_key = os.getenv("DEEPSEEK_API_KEY")
model_name = os.getenv("DEEPSEEK_MODEL")
embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5", base_url="http://localhost:11434"
)

# 设置 LLM
llm = DeepSeek(model=model_name, api_key=api_key)

In [76]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import AsyncQdrantClient, QdrantClient

client = QdrantClient(url="http://localhost:6333")
aclient = AsyncQdrantClient(url="http://localhost:6333")


In [79]:
client.delete_collection("hybrid_nodes")

True

In [77]:
collection_name = "hybrid_nodes"

In [78]:
if client.collection_exists(collection_name):
    print(1)
else:
    print(2)


1


In [83]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import load_index_from_storage

persist_dir = test_data_path / collection_name
all_nodes_docstore = SimpleDocumentStore()
all_nodes_docstore.add_documents(hybrid_all_nodes)


In [84]:
# 如果是传入了nodes类型，直接用类初始化，如果是docs则用from_documents初始化

vector_store_base = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name=collection_name,
)

In [85]:
hybrid_index = None
if client.collection_exists(collection_name):
    print(f"collection_name: '{collection_name}' 已存在")
    # 创建storage_context
    storage_context = StorageContext.from_defaults(
        persist_dir=persist_dir, vector_store=vector_store_base
    )
    hybrid_index = load_index_from_storage(
        storage_context, llm=llm, embed_model=embed_model
    )
    print("加载向量完成")
else:
    print(f"collection_name: '{collection_name}' 不存在，需要初始化")
    qdrant_storage_context = StorageContext.from_defaults(
        vector_store=vector_store_base,
        docstore=all_nodes_docstore,
    )
    hybrid_index = VectorStoreIndex(
        nodes=hybrid_all_nodes,
        embed_model=embed_model,
        llm=llm,
        storage_context=qdrant_storage_context,
        show_progress=True,
    )
    print("初始化本地向量完成")

collection_name: 'hybrid_nodes' 不存在，需要初始化


Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/705 [00:00<?, ?it/s]

初始化本地向量完成


### 数据持久化


DocStore与IndexStore的区别
1.  DocStore (文档存储)
- 存储原始文档节点的**实际内容**
- 包含完整的文本、元数据和节点ID
- 类似于"数据库"，保存所有节点的原始信息

2. IndexStore (索引存储)
- 存储索引的**结构信息**
- 保存索引类型、索引配置和索引之间的关系
- 维护索引的元数据，如索引名称、索引类型
- 包含指向docstore中文档的引用，但不直接存储文档内容

关系说明
两者关系类似于：
- **DocStore**: 图书馆中的所有书籍(实际内容)
- **IndexStore**: 图书馆的分类目录(如何组织和查找这些书籍)

所以当你需要：
- 保留节点唯一ID和内容 → 需要DocStore
- 保留索引结构和检索能力 → 需要IndexStore
- 完整恢复整个系统 → 需要全部存储组件

重要细节
- 从代码中可以看出，向量存储会按命名空间分开存储，每个都有单独的文件
- 持久化会保留完整的对象关系，包括各种引用和指针
- 即使不使用某些存储(如graph_store)，也会创建并持久化一个空的存储对象


#### StorageContext持久化

In [86]:
try:
    hybrid_index.storage_context.persist(persist_dir)
    print("持久化完成")
except Exception as e:
    print(e)


持久化完成


### Retrievers

- docstore 和 nodes的区别注意区分理解
- 如果用bm25加载从docstore，nodes会产生不一样的结果吗？

In [90]:
import jieba

stop_words_path = project_root / "readai/utils/baidu_stopwords.txt"
# jieba.load_userdict(str(stop_words_path))
custom_tokenizer = jieba.Tokenizer()
# 加载自定义词典
custom_tokenizer.load_userdict(str(stop_words_path))
# 使用自定义分词器进行分词
text = "我来到北京清华大学"
seg_list = custom_tokenizer.cut(text, cut_all=False)
print("/ ".join(seg_list))


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/70/hwlr8qxx6g751fyxb33mcys40000gn/T/jieba.cache
Loading model cost 0.263 seconds.
Prefix dict has been built successfully.


我/ 来到/ 北京/ 清华大学


In [91]:
# 加载stop_words为字符串列表
with open(stop_words_path) as f:
    stop_words = f.readlines()

stop_words = set(stop_words)
print(len(stop_words))

1396


In [172]:
# 如果加载是从docstore会一样吗？
bm25_retriever = BM25Retriever.from_defaults(
    nodes=hybrid_all_nodes,
    similarity_top_k=5,
    tokenizer=custom_tokenizer,
    stopwords=stop_words,
)
# bm25_retriever = BM25Retriever.from_defaults(nodes=hybrid_all_nodes, similarity_top_k=2)


In [94]:
sparse_nodes = bm25_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(len(sparse_nodes))
for n in sparse_nodes:
    display_source_node(n, source_length=1500)

4


**Node ID:** dc4529ef-133f-4283-a196-e238835caeaf<br>**Similarity:** 26.955977526355234<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** 87c78ce2-6bab-4488-b84b-b499c3e29187<br>**Similarity:** 26.268064015672522<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

**Node ID:** f4dfebef-af6a-4edb-89b7-c1b1e2e04e2d<br>**Similarity:** 26.214797279821816<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** 1ef984c1-b519-4884-b417-2c659e3b7631<br>**Similarity:** 24.495895670924263<br>**Text:** ” 刚开始尝试请他人重述我们的话时，可能会觉得有点不自然。因为在日常交流中，我们很少会提出这样的请求。当我强调请他人重述我们的话是一种重要的能力时，人们通常担心有人会这样反应：“你觉得我是聋子吗？<br>

#### small-to-big retriever

In [95]:
hybrid_all_nodes_dict = hybrid_index.storage_context.docstore.docs

In [134]:
from llama_index.core.retrievers import RecursiveRetriever

vector_retriever = hybrid_index.as_retriever(similarity_top_k=10)
small2big_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=hybrid_all_nodes_dict,
    verbose=True,
)

In [97]:
# 获取向量召回结果
dense_nodes = small2big_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)

Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 7b5a47cb-9add-42fd-a70f-b497ebbeefcf
Retrieving with query id 7b5a47cb-9add-42fd-a70f-b497ebbeefcf: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 26e36c51-f176-4d0c-919a-a31afc84bcee
Retrieving with query id 26e36c51-f176-4d0c-919a-a31afc84bcee: 在日常交流中，我们如何区分客观的观察和主观的评价？


In [98]:
for chunk_node in dense_nodes:
    display_source_node(chunk_node, source_length=1500)


**Node ID:** af90f2ee-62d7-4fbc-b60d-28cd2a88327d<br>**Similarity:** 0.6397337<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** 7b5a47cb-9add-42fd-a70f-b497ebbeefcf<br>**Similarity:** 0.63314337<br>**Text:** ” “不是这里，”男子回答，“我把钥匙落在巷子里了。” 见到警察很困惑，男子急忙解释：“但这里的光线要亮得多呢。” 我们都对人生拥有渴望。然而我发现，主导我们的文化却可能带我们偏离了方向。为此，我开发了非暴力沟通。它作为一种训练注意力的方式，让我们可以把注意力放在真正要去的地方。我渴望的是，在生命中活出慈悲之心，让由衷的相互给予，在你我之间流动。 那份慈悲心，也是我所说的“由衷的给予”，在我的朋友鲁斯·贝本梅尔（Ruth Bebermeyer）的这段歌词中得到展现： 你取之于我，是对我最好的馈赠。 请你知道，这种给予带给我快乐。 我给予绝非冀盼你偿还， 只因我想活出对你的爱。 也许，你的欣然接受， 就是对我的最佳馈赠。 我无法将两者分开。 你施于我，我给予我的接受； 你取之于我，是对我最好的馈赠。 ——“馈赠”（1978）来自鲁斯·贝本梅尔 唱片集《馈赠》 每当我们想要丰盈他人的生命，由衷给予他人时，我们是出于心中油然而生的喜悦之情。给予者和接受者都会从中获益。当给予是由衷的，而非出于害怕、内疚、羞愧或是渴求回报，接受者才能心无挂碍地享受这份馈赠。与此同时，给予者因为看到自己为他人的幸福做出了贡献，也会收获更多的自我肯定。 我们使用非暴力沟通，并不要求沟通的对方也懂得它，甚至无须对我们保有善意。只要我们遵循非暴力沟通的原则，发自内心地给予，并尽我们所能让对方知道这是我们唯一的动机，他们也会加入我们，最终双方必然能以善意相待。当然，这一切不一定会很快发生，但我相信，只要我们坚持非暴力沟通的原则与方法，友善之花终将在你我心中盛开。 非暴力沟通过程 要达到让双方都能发自内心地相互给予，我们需将注意力聚焦在四个方面——它们是非暴力沟通模式的四个要素。 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** 26e36c51-f176-4d0c-919a-a31afc84bcee<br>**Similarity:** 0.62933075<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

#### 使用HybridRetriever

In [162]:
from enum import Enum


class FUSION_MODES(str, Enum):
    """Enum for different fusion modes."""

    RECIPROCAL_RANK = "reciprocal_rerank"  # apply reciprocal rank fusion
    RELATIVE_SCORE = "relative_score"  # apply relative score fusion
    DIST_BASED_SCORE = "dist_based_score"  # apply distance-based score fusion
    SIMPLE = "simple"  # simple re-ordering of results based on original scores


In [181]:
from llama_index.core.retrievers import QueryFusionRetriever

fusion_retriever = QueryFusionRetriever(
    retrievers=[small2big_retriever, bm25_retriever],
    retriever_weights=[0.3, 0.7],
    mode=FUSION_MODES.RELATIVE_SCORE,
    verbose=True,
    similarity_top_k=10,
    llm=llm,
    num_queries=1,  # 不需要查询扩展,查询重写
)


In [200]:
nodes = fusion_retriever.retrieve("在日常交流中，我们如何区分客观的观察和主观的评价？")
print(len(nodes))
for node in nodes:
    display_source_node(node, source_length=1500)

Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？


2025-04-08 22:44:48.019 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 7b5a47cb-9add-42fd-a70f-b497ebbeefcf
Retrieving with query id 7b5a47cb-9add-42fd-a70f-b497ebbeefcf: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 26e36c51-f176-4d0c-919a-a31afc84bcee
Retrieving with query id 26e36c51-f176-4d0c-919a-a31afc84bcee: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 1ef4270e-5bcc-428b-b531-da0bce95a6df
Retrieving with query id 1ef4270e-5bcc-428b-b531-da0bce95a6df: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: ee702fc3-685b-442b-9f5a-84fe20693636
Retrieving with query id ee702fc3-685b-442b-9f5a-84fe20693636: 在日常交流中，我们如何区分客观的观察和主观的评价？
9


**Node ID:** dc4529ef-133f-4283-a196-e238835caeaf<br>**Similarity:** 0.7<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** 87c78ce2-6bab-4488-b84b-b499c3e29187<br>**Similarity:** 0.5042587662622552<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

...

**Node ID:** 7b5a47cb-9add-42fd-a70f-b497ebbeefcf<br>**Similarity:** 0.24195365920254142<br>**Text:** ” “不是这里，”男子回答，“我把钥匙落在巷子里了。” 见到警察很困惑，男子急忙解释：“但这里的光线要亮得多呢。” 我们都对人生拥有渴望。然而我发现，主导我们的文化却可能带我们偏离了方向。为此，我开发了非暴力沟通。它作为一种训练注意力的方式，让我们可以把注意力放在真正要去的地方。我渴望的是，在生命中活出慈悲之心，让由衷的相互给予，在你我之间流动。 那份慈悲心，也是我所说的“由衷的给予”，在我的朋友鲁斯·贝本梅尔（Ruth Bebermeyer）的这段歌词中得到展现： 你取之于我，是对我最好的馈赠。 请你知道，这种给予带给我快乐。 我给予绝非冀盼你偿还， 只因我想活出对你的爱。 也许，你的欣然接受， 就是对我的最佳馈赠。 我无法将两者分开。 你施于我，我给予我的接受； 你取之于我，是对我最好的馈赠。 ——“馈赠”（1978）来自鲁斯·贝本梅尔 唱片集《馈赠》 每当我们想要丰盈他人的生命，由衷给予他人时，我们是出于心中油然而生的喜悦之情。给予者和接受者都会从中获益。当给予是由衷的，而非出于害怕、内疚、羞愧或是渴求回报，接受者才能心无挂碍地享受这份馈赠。与此同时，给予者因为看到自己为他人的幸福做出了贡献，也会收获更多的自我肯定。 我们使用非暴力沟通，并不要求沟通的对方也懂得它，甚至无须对我们保有善意。只要我们遵循非暴力沟通的原则，发自内心地给予，并尽我们所能让对方知道这是我们唯一的动机，他们也会加入我们，最终双方必然能以善意相待。当然，这一切不一定会很快发生，但我相信，只要我们坚持非暴力沟通的原则与方法，友善之花终将在你我心中盛开。 非暴力沟通过程 要达到让双方都能发自内心地相互给予，我们需将注意力聚焦在四个方面——它们是非暴力沟通模式的四个要素。 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** 26e36c51-f176-4d0c-919a-a31afc84bcee<br>**Similarity:** 0.20837284612471327<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

**Node ID:** 1ef4270e-5bcc-428b-b531-da0bce95a6df<br>**Similarity:** 0.11548133185753703<br>**Text:** 菲 尔：是的。 马歇尔：（当我清楚地了解了他的观察、感受、需要以及请求后，我问自己是否愿意满足他的请求）是的，这个方法对我来说也是不容易的。在接下来的时间里，你也许会听到我讲述那些抓狂的时刻，有时甚至完全做不到。而当我真的能够运用非暴力沟通时，就可以和他人建立紧密的连结。因此，尽管有许多困难，我依然愿意坚持尝试这个方法。 练习一：观察还是评论？ 请完成以下练习，看看你是否可以熟练区分观察和评论。请标出那些只是描述观察而不带有评论的句子。 1. “昨天，约翰无缘无故冲我发脾气。” 2. “昨天晚上，南希一边看电视、一边咬指甲。” 3. “会议上我没有听到桑姆询问我的意见。” 4. “我的父亲是个好人。” 5. “詹尼斯花在工作上的时间太多了。” 6. “亨利很强势。” 7. “这周，潘每天第一个来排队。” 8. “我的儿子经常不刷牙。” 9. “卢克说，我穿黄色衣服不好看。” 10.“姑姑和我说话时一直在抱怨。” 以下是我对练习一的回应： 1. 如果你认为这句话是观察，我们的意见不同。我认为“无缘无故”是评论。此外，我认为说约翰“发脾气”也是评论。他也许感到受伤、害怕、伤心……不带有评论的观察可以是：“约翰告诉我他在生气。”或者是：“约翰用拳头砸了一下桌子。” 2. 如果你认为这句话是观察，我们的意见一致，这是一个不带评论的观察。 3. 如果你认为这句话是观察，我们的意见一致，这是一个不带评论的观察。 4. 如果你认为这句话是观察，我们的意见不同。我认为“好人”是评论。不带评论的观察可以是：“过去25年来，我的父亲将他收入的十分之一捐给了慈善事业。” 5. 如果你认为这句话是观察，我们的意见不同。我认为“太多”是评论。不带评论的观察可以是：“这周詹尼斯在办公室工作了60小时以上。” 6. 如果你认为这句话是观察，我们的意见不同。我认为“强势”是评论。不带评论的观察可以是：“亨利在妹妹切换电视频道时撞了她一下。” 7. 如果你认为这句话是观察，我们的意见一致，这是一个不带评论的观察。 8.<br>

**Node ID:** ee702fc3-685b-442b-9f5a-84fe20693636<br>**Similarity:** 0.0<br>**Text:** ·我记得你从来没有写信给我。 然而有时，这些词是言过其实的表达，混淆了观察和评论： ·你总是在忙。 ·在需要她的时候，她永远都不在。 这样的表达经常会引发他人的逆反心理，而非慈悲之情。 “经常”“很少”这样的词也可能混淆观察和评论。 小结 非暴力沟通的第一个要素是区分观察与评论。当我们在观察中夹杂着自己的评论时，他人往往会认为我们在批评他们，并因而产生抗拒的心理。非暴力沟通是一种动态的语言，它不鼓励人们做静态、一概而论的陈述，而是提倡我们在描述观察时，清楚地说出特定的时间和情境。例如“汉克在过去20场球赛中未进一球”，而非“汉克是名糟糕的足球运动员”。 非暴力沟通实例 “我们遇到过的最傲慢的演讲者！” 以下对话来自我主持的一场工作坊。在讲解了约半小时后，我停下来请学员谈谈他们的心得。这时，有人举起手说：“你是我们遇到过的最傲慢的演讲者！” 当听到有人这样说我，我的回应方式可以有几种选择。其中一种是认为这确实是我的问题。我知道，一旦我想讨好对方、为自己辩护或开脱，就可以将错误归咎到自己身上。另一种方式是认为对方在攻击我而开始反击，这也是我过去时常选择的方式。而这一次，我选择了第三种方式，既没有辩解也没有指责对方，而是将注意力放在他言语背后发生了什么。 马歇尔：（猜测他的观察）你这么说，是针对我连续讲了半小时才给你们机会发表意见吗？ 菲 尔：不是，你把事情说得太简单了。 马歇尔：（请他澄清）那是不是因为我没有提到非暴力沟通对有些人来说用起来会很困难？ 菲 尔：不，不是有些人，而是你！ 马歇尔：所以，你这么说是因为我没有提到，有时我也很难运用非暴力沟通，是吗？ 菲 尔：正是。 马歇尔：你感到不高兴，因为你希望我能告诉你们，我自己在使用这个方法时会遇到的困难？ 菲 尔：（犹豫了一会儿）没错。 马歇尔：（当我能体会到他的感受和需要时，我感到放松了许多，我开始关注他的请求）你是不是希望此时我能够承认，我自己在运用这个方法的时候，也会遇到一些困难？ 菲 尔：是的。<br>

**Node ID:** 1ef984c1-b519-4884-b417-2c659e3b7631<br>**Similarity:** 0.0<br>**Text:** ” 刚开始尝试请他人重述我们的话时，可能会觉得有点不自然。因为在日常交流中，我们很少会提出这样的请求。当我强调请他人重述我们的话是一种重要的能力时，人们通常担心有人会这样反应：“你觉得我是聋子吗？<br>

In [101]:
from llama_index.core.settings import Settings

Settings.llm = llm

query_engine = RetrieverQueryEngine(fusion_retriever)
response = query_engine.query("在日常交流中，我们如何区分客观的观察和主观的评价？")

2025-04-08 13:44:00.002 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 7b5a47cb-9add-42fd-a70f-b497ebbeefcf
Retrieving with query id 7b5a47cb-9add-42fd-a70f-b497ebbeefcf: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 26e36c51-f176-4d0c-919a-a31afc84bcee
Retrieving with query id 26e36c51-f176-4d0c-919a-a31afc84bcee: 在日常交流中，我们如何区分客观的观察和主观的评价？


In [102]:
print(response)

在日常交流中，区分客观观察和主观评价的关键在于：客观观察是描述具体发生的事实或行为，而不加入个人判断或解读；主观评价则包含了个人的观点、感受或对他人动机的推测。

例如：
- 观察："会议比预定时间延长了20分钟"
- 评价："会议组织得很糟糕"

要清晰地区分两者，可以注意以下几点：
1. 使用具体的行为描述而非概括性词语
2. 避免使用带有判断性质的形容词
3. 注意区分事实描述和个人感受
4. 警惕使用"总是"、"从不"等绝对化词语（除非确实观察到重复模式）

有效的沟通需要保持这种区分，因为纯粹的观察更容易被接受，而夹杂评价的表达容易引发对方的防御心理。当我们需要表达不同意见时，先陈述观察到的事实，再表达自己的感受和需求，这样的沟通方式往往更有效。


#### 对比其他单一检索器

In [169]:
simple_vector_retriever = hybrid_index.as_retriever(similarity_top_k=10)

In [146]:
bm25_query_engine = RetrieverQueryEngine(bm25_retriever)
response = bm25_query_engine.query("在日常交流中，我们如何区分客观的观察和主观的评价？")
print(response)


在日常交流中，区分客观观察和主观评价的关键在于：观察是描述具体事实或行为，而评价则包含个人判断或标签化的语言。例如，"每次会议你都迟到十分钟"是观察，而"你总是没有时间观念"则是评价。有效的沟通需要清晰表达事实性信息，避免将个人解读与事实混为一谈。同时，要注意某些词语如"总是""从不"可能隐含评价，即使表面看似客观描述。保持语言的具体性和时效性，避免绝对化表述，有助于更准确地传达观察而非批评。


In [147]:
small2big_engine = RetrieverQueryEngine(small2big_retriever)
response = small2big_engine.query("在日常交流中，我们如何区分客观的观察和主观的评价？")
print(response)


Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 7b5a47cb-9add-42fd-a70f-b497ebbeefcf
Retrieving with query id 7b5a47cb-9add-42fd-a70f-b497ebbeefcf: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 26e36c51-f176-4d0c-919a-a31afc84bcee
Retrieving with query id 26e36c51-f176-4d0c-919a-a31afc84bcee: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 1ef4270e-5bcc-428b-b531-da0bce95a6df
Retrieving with query id 1ef4270e-5bcc-428b-b531-da0bce95a6df: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: ee702fc3-685b-442b-9f5a-84fe20693636
Retrieving with query id ee702fc3-685b-442b-9f5a-84fe20693636: 在日常交流中，我们如何区分客观的观察和主观的评价？
在日常交流中，区分客观观察和主观评价的关键在于是否包含个人判断或情绪色彩。客观观察应仅描述具体发生的事实，而不添加解释或标签。例如：

1. 使用可验证的具体行为描述（如"这周潘每天第一个来排队"）
2. 避免使用概括性标签（如"好人""强势"等抽象评价）
3. 注意时间限定（如"昨天

## Evaluation

In [103]:
import json
import random

import pandas as pd
from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

from readai.components.prompts import QA_GENERATE_PROMPT_TMPL_ZH

In [104]:
QA_GENERATE_PROMPT_TMPL_ZH = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a university professor. Your task is to set {num_questions_per_chunk} questions for the upcoming Chinese quiz.
Questions throughout the test should be diverse. Questions should not contain options or start with Q1/Q2.
Questions must be written in Chinese. The expression must be concise and clear. 
It should not exceed 15 Chinese characters. Words such as "这", "那", "根据", "依据" and other punctuation marks 
should not be used. Abbreviations may be used for titles and professional terms.
"""

In [ ]:
print(QA_GENERATE_PROMPT_TMPL_ZH)

In [106]:
for k, v in list(hybrid_index.docstore.docs.items())[:5]:
    print(k, v.text)

045b921e-231e-4a73-a426-b87f95c05aed 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。
c25c4cd4-ccc8-41dd-a4ae-6e24e2049c25 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！
9d12db15-bf0c-4245-834f-163a0c5c73eb 言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。
9c71d44b-14c5-4169-870e-405d8c1baba3 这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦
f3b5de80-9d4f-4444-8b1c-7c3c5e026e0b 伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。


In [107]:
print(len(hybrid_all_nodes))

2753


In [113]:
test_hybrid_nodes = random.sample(hybrid_all_nodes, 50)
print("测试节点数", len(test_hybrid_nodes))
print(
    f"文本长度：{len(test_hybrid_nodes[0].text)}，节点id：{test_hybrid_nodes[0].node_id}"
)

测试节点数 50
文本长度：83，节点id：7adc1a72-f1c6-4127-a5b2-3082e3686060


In [114]:
eval_dataset_zh = generate_question_context_pairs(
    test_hybrid_nodes, llm=llm, qa_generate_prompt_tmpl=QA_GENERATE_PROMPT_TMPL_ZH
)

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [06:58<00:00,  8.36s/it]


In [115]:
test_qa_datasets = test_data_path / "feibaoli_hybrid_eval_dataset50_zh.json"

with open(test_qa_datasets, "w") as f:
    json.dump(eval_dataset_zh.model_dump(), f, indent=4, ensure_ascii=False)

In [245]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=fusion_retriever
)

In [188]:
sample_id, sample_query = list(eval_dataset_zh.queries.items())[2]
print(sample_id)
sample_expected = eval_dataset_zh.relevant_docs[sample_id]
print(sample_expected)
eval_result = retriever_evaluator.evaluate(sample_query, expected_ids=sample_expected)
print(eval_result)

2025-04-08 19:53:09.955 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


3ebf5c4b-464c-44fe-af6c-327ea1848e88
['d4f1dbd6-c88c-4144-b89d-890668eac513']
Retrieving with query id None: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 56849c71-c844-4a50-ac5d-4d70f32bcc70
Retrieving with query id 56849c71-c844-4a50-ac5d-4d70f32bcc70: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: cd036189-1e46-44dc-afcd-043baa4374f9
Retrieving with query id cd036189-1e46-44dc-afcd-043baa4374f9: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 70c7c895-14bc-40e1-86ca-6250441733ce
Retrieving with query id 70c7c895-14bc-40e1-86ca-6250441733ce: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 5fd22041-b5d8-431d-bead-ce7740c10e71
Retrieving with query id 5fd22041-b5d8-431d-bead-ce7740c10e71: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: cd280302-64f2-4bee-9aa9-67f2000f57e2
Retrieving with query id cd280302-64f2-4bee-9aa9-67f2000f57e2: 调解过程中，双方对"埃及黑人"这一身份标签

In [189]:
nodes = fusion_retriever.retrieve(sample_query)

2025-04-08 19:53:18.364 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 56849c71-c844-4a50-ac5d-4d70f32bcc70
Retrieving with query id 56849c71-c844-4a50-ac5d-4d70f32bcc70: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: cd036189-1e46-44dc-afcd-043baa4374f9
Retrieving with query id cd036189-1e46-44dc-afcd-043baa4374f9: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 70c7c895-14bc-40e1-86ca-6250441733ce
Retrieving with query id 70c7c895-14bc-40e1-86ca-6250441733ce: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 5fd22041-b5d8-431d-bead-ce7740c10e71
Retrieving with query id 5fd22041-b5d8-431d-bead-ce7740c10e71: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: cd280302-64f2-4bee-9aa9-67f2000f57e2
Retrieving with query id cd280302-64f2-4bee-9aa9-67f2000f57e2: 调解过程中，双方对"埃及黑人"这一身份标签的不同理解如何影响了冲突的解决进程？
Retrieved node with id, entering: 773a48a8-4fb6-4362-8b40-c

通过下面的单例测试发现，问题问的比较复杂，虽然确实召回了对应的节点，但这个节点不足以回答内容，所以这里的mrr分数并不能客观反映我的检索效果，那我该如何优化？我需要优化hit_rate,mrr吗？
- 这涉及如何构造合理测试集的问题

In [194]:
for node in nodes:
    if node.node_id in sample_expected:
        print("命中节点")
        print(node.text)
    print(node.node_id)
    print(node.score)
    print("-" * 100)

e9a2ec2c-89d5-42f9-b99d-26ff02d5b8bc
0.7
----------------------------------------------------------------------------------------------------
97f61a65-56a7-4ec4-a0aa-89b78a6068a2
0.6516779763107587
----------------------------------------------------------------------------------------------------
56849c71-c844-4a50-ac5d-4d70f32bcc70
0.3
----------------------------------------------------------------------------------------------------
cd036189-1e46-44dc-afcd-043baa4374f9
0.09414518604496687
----------------------------------------------------------------------------------------------------
70c7c895-14bc-40e1-86ca-6250441733ce
0.08792342142340212
----------------------------------------------------------------------------------------------------
命中节点
有一次，我花了3天时间调停两派人马之间的冲突。其中一派称自己为“埃及黑人”，另一派来自东圣路易斯警察局。一个月内，一方死了2人，另一方死了1人。那3天中，我努力帮助双方相互听见并化解分歧。
d4f1dbd6-c88c-4144-b89d-890668eac513
0.0684139114854703
---------------------------------------------------------------------------------------

In [195]:
print(nodes[0].text)

掌握进度：追踪 在调解过程中，我们务必做好“追踪”进程的工作——仔细注意人们所说的话，确保双方都有机会表达自己的需要，聆听对方的需要并提出请求。此外，我们还要“追踪”——注意其中一方刚才说到哪里，以便在另一方说完后，回到前者所说的内容。 要这样做并不容易，尤其在双方争执不下、情绪激动时。此时，如果能在白板或挂纸上将上一位讲话者所表述的要义（感受或需要）记录下来，会颇有帮助。 用视觉的形式追踪进程也能让双方确保自己的需要会被听见。调解过程中经常会发生这样的状况：调解人还没来得及引导一方充分表达完自己的需要，另一方就抢着发言了。花一些时间记录下需要，并且让在场的每一个人都看得到，这会让聆听的一方感到踏实，相信自己的需要会得到处理。这样，每个人就都能更专心地听对方说话了。 5. 聚焦于当下 调解人需要为调解过程注入的另一个重要品质是——对当下的觉察：此刻谁需要什么？人们此刻的请求是什么？要保持这样的觉察，需要大量练习如何安住在当下，而我们大部分人从未受过这方面的训练。 在调解过程中，我们会听到人们有许多讨论，关于过去所发生的事件以及未来所期望的改变。但唯有把握此时此刻才能化解冲突，因此，当下才是我们需要关注的焦点。 6. 推动对话进程 调解人的另一项任务是避免对话陷入泥潭。人们往往认为只要把同样的话多说几遍，别人就能够理解并且按照他们的意思去做，因此，对话很容易陷入原地打转。 为了推动对话进程，调解人需要提出有效的问题，并且在必要时刻维持好对话的节奏，甚至加快步调。有一次，我受邀在一个小镇主持工作坊，主办方问我是否可以帮助他调和一桩家庭内财产分配纠纷。我答应了，同时也意识到我在课余只有3、4个小时的空档来做这件事。 这场纠纷的核心是，一个拥有一大片农场的男子即将退休，他的两个儿子为了财产分配陷入了剧烈争吵。两人虽然都住在农场的一端，彼此距离很近，但已经有8年不曾交谈了。我会见了这对兄弟以及他们的妻子和姐妹。8年来，所有人都卷入这桩复杂的法律纠纷，非常痛苦。 为了让调解工作取得进展，并且不耽误我的工作坊，我不得不加快调解的进程。


#### 检测query生成质量

In [119]:
def get_node_by_query_id(dataset: EmbeddingQAFinetuneDataset, query_id):
    """根据query_id查找对应的节点ID和文本内容

    参数:
        dataset: 生成的数据集对象
        query_id: 问题的唯一标识

    返回:
        包含节点ID和文本内容的字典列表
    """
    # 检查query_id是否存在
    if query_id not in dataset.relevant_docs:
        print(f"query_id {query_id} not in dataset.relevant_docs")
        return []

    print(f"query_id {query_id} in dataset.relevant_docs")
    # 输出query_id对应的query
    print(f"对应的query: {dataset.queries[query_id]}")

    # 获取相关节点ID列表
    node_ids = dataset.relevant_docs[query_id]

    # 构建结果
    results = []
    for node_id in node_ids:
        if node_id in dataset.corpus:
            results.append({"node_id": node_id, "text": dataset.corpus[node_id]})

    print("该测试query相关的节点以及内容如下：")
    for node in results:
        print(f"节点ID: {node['node_id']}")
        print(f"节点内容: {node['text']}")
    return results


In [94]:
query_id = "4ec0062d-03f9-43cc-8f3a-e7ca0294d86e"
res_nodes = get_node_by_query_id(eval_dataset_zh, query_id)

query_id 4ec0062d-03f9-43cc-8f3a-e7ca0294d86e in dataset.relevant_docs
对应的query: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
该测试query相关的节点以及内容如下：
节点ID: e57510a9-e142-4b21-af6a-3eff2905ac1a
节点内容: 相反，我们被教育成乖孩子、好妈妈、好爸爸。如果我们要成为那样的好人，就得习惯压抑自己。抑郁，是我们做一个“好人”的“代价”。如果你想要让自己好起来，我希望你能清楚地说出，为了让你的生活变得美好，你希望他人做些什么。 来访者：我只是希望有人能爱我。这并非完全不现实，不是吗？ 马歇尔：这是个好的开始。现在我想请你清晰地说出，他人怎么做才能满足你爱的需要？例如，我此刻可以做些什么？ 来访者：哦，你知道的…… 马歇尔：我不确定我知道。我想请你告诉我，为了让你得到你想要的爱，你希望我或者他人怎样做？ 来访者：这很难说清楚。 马歇尔：是的，提出清晰的请求是困难的。但是请想一想，如果我们讲不清楚自己想要什么，别人要回应我们的请求该会有多困难！ 来访者：我开始明白，我希望别人做什么来爱我了。不过，我不太好意思说出来。 马歇尔：是的，这经常是令人尴尬的。那么，你希望我或者他人怎么做？ 来访者：我认真思考了，当我说希望得到爱时，我实际想要的是，即使我还不知道我的需要，你也能猜到我想要什么，并且我希望你一直都这样。 马歇尔：我很感激你做了澄清，我也希望你已经明白了，找到这样一个人用这样的方式来爱你是不可能的。 经过这样的对话，许多来访者才发现，他们之所以感到沮丧和压抑，很大程度上是因为他们不清楚自己希望他人如何来满足自己的需要。 有意识地提出请求 有的时候，表达清晰的请求也许无需过多言语。例如，你正在厨房里，在客厅看电视的妹妹喊道：“我口渴。”在这种情况下，多半她在请求你给她从厨房倒一杯水。 但有时，我们在表达自己的不快时，却误以为对方能明白我们想要什么。例如，一位妻子对丈夫说：“我不是让你在路上带一些黄油和洋葱回来吗？你怎么忘记了？真气人！”也许，妻子的意思是希望丈夫再去一趟商店买东西，但丈夫却很有可能将太太的话视作对他的指责。如果我们只是表达自己的感受，别人可能并不清楚我们想要他们做什

In [316]:
# 对单个节点生成query
num_questions_per_chunk = 5
for node in res_nodes:
    query = QA_GENERATE_PROMPT_TMPL_ZH.format(
        context_str=node["text"], num_questions_per_chunk=num_questions_per_chunk
    )
    print("生成query：", query)
    response = llm.complete(query)
    # result = str(response).strip()
    result = str(response).strip().split("\n")
    print("生成的问题：\n")
    for question in result:
        print(question)

生成query： ##任务：根据以下书籍片段内容，生成具有多维度的讨论性和理解性问题，帮助用户深入理解该段内容。
##Examples:

input_case1:
上下文内容：
---------------------
在《论语》中，孔子提出了“仁”的概念，认为“仁”是人与人之间相互关爱、尊重和理解的基础。他强调，只有具备“仁”的品质，才能真正实现社会的和谐与稳定。
---------------------
请根据上下文生成2个问题
output:
孔子所倡导的“仁”在现代社会中具有怎样的现实意义？
如何理解“仁”是实现社会和谐与稳定的基础？

input_case2:
上下文内容：
---------------------
《物种起源》中，达尔文提出了自然选择学说，认为生物的进化是通过自然选择的过程实现的。他指出，适应环境的个体更容易生存和繁衍后代，从而推动了物种的进化。
---------------------
请根据上下文生成1个问题
output:
适应环境的个体更容易生存和繁衍后代，这一观点如何解释生物多样性的形成？

##下面是用户输入：
上下文内容：
---------------------
我记得最后一天开车回家的路上，我和自己说：“这辈子再也不想身处这样的冲突中了。” 没想到一进家门，我就看到孩子们正在打架。那时，我已经没有力气去同理他们了，便用非暴力沟通的方式大声地呐喊道：“我现在很难受！真的不想来管你们打架的事情，我只想要安静一会儿！”听到这番话，我年仅9岁的大儿子停了下来，看着我问道：“你愿意和我们说说吗？” 我发现，如果我们能坦诚并且不带指责地谈论自己的痛苦，即使对方也处于苦痛之中，有时也能够听见我们的需要。在前述的例子中，我不想冲着孩子大喊：“这怎么回事？你们就不能乖一点吗？我在外面辛苦了一整天才回到家啊！”我也不想用任何其他方式来暗示对方错了。“非暴力呐喊”就是提醒他人注意我在当下的痛苦和迫切的需要。 不过，如果对方也处于强烈的情绪中，无法听见我们，也不肯放过我们，而同时，同理“急救”和“非暴力呐喊”都无法帮助我们的话，我们的另一种选择就是：暂时离开现场，得到自己所需的同理心，等到心境调整好了之后再回去。 小结 所谓“同理”就是带着尊重来理解他人的经历。我们常常有强烈的冲动想给他人建议或安慰，或是解释自己的立场和感受。同理则

### 全测试集评估

In [167]:
eval_results = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)

  0%|          | 0/100 [00:00<?, ?it/s]2025-04-08 15:39:56.404 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-08 15:39:56.459 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-08 15:39:56.514 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 如果博尔特最终未能戒烟，这段对话中体现的友谊关系可能会面临哪些潜在挑战或变化？
Retrieved node with id, entering: 024a9f09-05ec-4745-a582-bb74c1b3754f
Retrieving with query id 024a9f09-05ec-4745-a582-bb74c1b3754f: 如果博尔特最终未能戒烟，这段对话中体现的友谊关系可能会面临哪些潜在挑战或变化？
Retrieved node with id, entering: 0fa8eb23-1aa8-408b-b8f9-5878542ca908
Retrieving with query id 0fa8eb23-1aa8-408b-b8f9-5878542ca908: 如果博尔特最终未能戒烟，这段对话中体现的友谊关系可能会面临哪些潜在挑战或变化？
Retrieving with query id None: 如果翻译始终无法突破礼貌的界限，可能会对调解结果产生哪些潜在负面影响？
Retrieved node with id, entering: c2c66937-abb3-4c45-a1e0-38a994926174
Retrieving with query id c2c66937-abb3-4c45-a1e0-38a994926174: 如果翻译始终无法突破礼貌的界限，可能会对调解结果产生哪些潜在负面影响？
Retrieved node with id, entering: f4020f3c-35db-481b-b766-1419044c9c62
Retrieving with query id f4020f3c-35db-481b-b766-1419044c9c62: 如果翻译始终无法突破礼貌的界限，可能会对调解结果产生哪些潜在负面影响？
Retrieved node with id, entering: cd036189-1e46-44dc-afcd-043baa4374f9
Retrieving with query id cd036189-1e46-44dc-afcd-043baa4374f9: 如果翻译始终无法突破礼貌的界限，可能会对调解结果产生哪些潜在负面影响？
Retr

  1%|          | 1/100 [00:00<00:24,  4.12it/s]2025-04-08 15:39:56.615 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-08 15:39:56.675 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  3%|▎         | 3/100 [00:00<00:10,  9.05it/s]2025-04-08 15:39:56.724 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 5fee85ad-5af3-43fe-8cc6-6a6702763064
Retrieving with query id 5fee85ad-5af3-43fe-8cc6-6a6702763064: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e
Retrieving with query id 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 005ec9b0-fcaf-4ac7-abac-af136cffd927
Retrieving with query id 005ec9b0-fcaf-4ac7-abac-af136cffd927: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 5fd22041-b5d8-431d-bead-ce7740c10e71
Retrieving with query id 5fd22041-b5d8-431d-bead-ce7740c10e71: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 57af29f0-d5e6-413e-ac3b-425af8b634ff
Retrieving with query id 57af29f0-d5e6-413e-ac3b-425af8b634ff: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性

2025-04-08 15:39:56.777 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  5%|▌         | 5/100 [00:00<00:07, 12.48it/s]2025-04-08 15:39:56.830 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-08 15:39:56.881 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  7%|▋         | 7/100 [00:00<00:06, 14.77it/s]2025-04-08 15:39:56.929 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


...

Retrieving with query id None: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: 6d109628-b62c-4b25-b6e3-aee968b76428
Retrieving with query id 6d109628-b62c-4b25-b6e3-aee968b76428: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: 9a79abbd-41f2-4164-a3f2-1526d4e40363
Retrieving with query id 9a79abbd-41f2-4164-a3f2-1526d4e40363: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: c2c66937-abb3-4c45-a1e0-38a994926174
Retrieving with query id c2c66937-abb3-4c45-a1e0-38a994926174: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: 8ad8e841-7389-44df-8a97-f312bb4bb0c0
Retrieving with query id 8ad8e841-7389-44df-8a97-f312bb4bb0c0: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: 268fc4c6-0d99-4363-b503-75330fc2d7e0
Retrieving with query id 268fc4c6-0d99-4363-b503-75330fc2d7e0: 如果当时讲述者使用了“可是”来反驳愤怒者，根据片段中的逻辑推测可能会引发怎样的后果？
Retrieved node with id, entering: 3f10573

2025-04-08 15:40:01.375 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
 93%|█████████▎| 93/100 [00:05<00:00, 18.77it/s]2025-04-08 15:40:01.430 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-08 15:40:01.484 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
 95%|█████████▌| 95/100 [00:05<00:00, 18.69it/s]2025-04-08 15:40:01.539 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: 678f652b-5d93-443f-ac4a-08a0ef80292e
Retrieving with query id 678f652b-5d93-443f-ac4a-08a0ef80292e: 在对话中提到的"更大的困难"可能涉及哪些非暴力沟通实践中的深层次挑战，与之前描述的"棘手情况"相比有何本质区别？
Retrieved node with id, entering: 7e01fa91-617f-4652-b129-db38a5bcf3e2
Retrieving with query id 7e01fa91-617f-4652-b129-db38a5bcf3e2: 在对话中提到的"更大的困难"可能涉及哪些非暴力沟通实践中的深层次挑战，与之前描述的"棘手情况"相比有何本质区别？
Retrieved node with id, entering: f9ea8138-a889-456c-8ea6-7d8bf7563771
Retrieving with query id f9ea8138-a889-456c-8ea6-7d8bf7563771: 在对话中提到的"更大的困难"可能涉及哪些非暴力沟通实践中的深层次挑战，与之前描述的"棘手情况"相比有何本质区别？
Retrieved node with id, entering: e327daa0-0785-4d76-a805-8b512ef6dcac
Retrieving with query id e327daa0-0785-4d76-a805-8b512ef6dcac: 在对话中提到的"更大的困难"可能涉及哪些非暴力沟通实践中的深层次挑战，与之前描述的"棘手情况"相比有何本质区别？
Retrieved node with id, entering: 57af29f0-d5e6-413e-ac3b-425af8b634ff
Retrieving with query id 57af29f0-d5e6-413e-ac3b-425af8b634ff: 在对话中提到的"更大的困难"可能涉及哪些非暴力沟通实践中的深层次挑战，与之前描述的"棘手情况"相比有何本质区别？
Retrieved node with id, entering: 137fef6e-cf

2025-04-08 15:40:01.592 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
100%|██████████| 100/100 [00:05<00:00, 18.70it/s]

Retrieved node with id, entering: 1e546c41-ee85-47d8-a9ef-14370198d105
Retrieving with query id 1e546c41-ee85-47d8-a9ef-14370198d105: 为什么像“你没有听明白我的话”这样的表述容易被误解为指责，而皮特的坦率回应却得到了老师的谢意？
Retrieved node with id, entering: 59d54651-c161-45c6-800f-97966e6dacba
Retrieving with query id 59d54651-c161-45c6-800f-97966e6dacba: 为什么像“你没有听明白我的话”这样的表述容易被误解为指责，而皮特的坦率回应却得到了老师的谢意？
Retrieving with query id None: 为什么在医护人员评估病情不严重的情况下，老太太仍持续表达消极厌世情绪，这反映了生理治疗与心理需求之间怎样的脱节？
Retrieved node with id, entering: 22182f51-fdb9-4351-b147-cf14ed79dc3d
Retrieving with query id 22182f51-fdb9-4351-b147-cf14ed79dc3d: 为什么在医护人员评估病情不严重的情况下，老太太仍持续表达消极厌世情绪，这反映了生理治疗与心理需求之间怎样的脱节？
Retrieved node with id, entering: 94807d05-0b0c-4165-8b54-175bc2ea93fa
Retrieving with query id 94807d05-0b0c-4165-8b54-175bc2ea93fa: 为什么在医护人员评估病情不严重的情况下，老太太仍持续表达消极厌世情绪，这反映了生理治疗与心理需求之间怎样的脱节？
Retrieved node with id, entering: 1d9b4cb4-fce3-40c1-8e5b-bd457d1ecc60
Retrieving with query id 1d9b4cb4-fce3-40c1-8e5b-bd457d1ecc60: 为什么在医护人员评估病情不严重的情况下，老太太仍持续表达消极厌世

In [121]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)
    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [144]:
display_results("Hybrid_retriever", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,Hybrid_retriever,0.58,0.238897,0.059984,0.58,0.238897,0.321467


In [180]:
display_results(
    "Hybrid_rerank_weights", eval_results
)  # set weights = [0.3, 0.7] mode=FUSION_MODES.RELATIVE_SCORE

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,Hybrid_rerank_weights,0.62,0.368052,0.063984,0.62,0.368052,0.429998


#### 测试其他检索器

In [173]:
bm25_retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=bm25_retriever
)
small2big_retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=small2big_retriever
)
simple_vector_retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=simple_vector_retriever
)


In [174]:
bm25_eval_results = await bm25_retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)
display_results("bm25_retriever", bm25_eval_results)

100%|██████████| 100/100 [00:00<00:00, 207.30it/s]


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,bm25_retriever,0.53,0.379667,0.129333,0.53,0.379667,0.416866


In [150]:
small2big_eval_results = await small2big_retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)
display_results("small2big_retriever", small2big_eval_results)

  0%|          | 0/100 [00:00<?, ?it/s]

Retrieving with query id None: 为什么角色扮演的方式可能比直接解释更能帮助对方理解有效沟通的重要性？
Retrieved node with id, entering: cd036189-1e46-44dc-afcd-043baa4374f9
Retrieving with query id cd036189-1e46-44dc-afcd-043baa4374f9: 为什么角色扮演的方式可能比直接解释更能帮助对方理解有效沟通的重要性？
Retrieved node with id, entering: f2932829-0d06-48d4-bdd6-05309f04664a
Retrieving with query id f2932829-0d06-48d4-bdd6-05309f04664a: 为什么角色扮演的方式可能比直接解释更能帮助对方理解有效沟通的重要性？
Retrieving with query id None: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: 14efc9e9-3bcf-41e1-9975-f002a784d3ea
Retrieving with query id 14efc9e9-3bcf-41e1-9975-f002a784d3ea: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: 26e36c51-f176-4d0c-919a-a31afc84bcee
Retrieving with query id 26e36c51-f176-4d0c-919a-a31afc84bcee: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: fbea843d-faf5-40f4-b2c5-7cbf9cfb64bd
Retrieving with query id fbea843d-faf5-40f4-b2c5-7cbf9cfb64bd: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区

100%|██████████| 100/100 [00:04<00:00, 22.01it/s]

Retrieved node with id, entering: a8f63fdd-3893-4761-be42-114957027a8b
Retrieving with query id a8f63fdd-3893-4761-be42-114957027a8b: 如果监狱长官立即拒绝了该囚犯的请求而非拖延回复，你认为囚犯的情绪反应会有怎样的不同？这种差异揭示了什么关于人类愤怒触发机制的心理规律？
Retrieved node with id, entering: 7c1f6052-2855-4d13-a71d-ccbffd7c0934
Retrieving with query id 7c1f6052-2855-4d13-a71d-ccbffd7c0934: 如果监狱长官立即拒绝了该囚犯的请求而非拖延回复，你认为囚犯的情绪反应会有怎样的不同？这种差异揭示了什么关于人类愤怒触发机制的心理规律？
Retrieving with query id None: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 5fee85ad-5af3-43fe-8cc6-6a6702763064
Retrieving with query id 5fee85ad-5af3-43fe-8cc6-6a6702763064: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e
Retrieving with query id 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e: 作者在致谢中提到的多位学者对其理论发展的影响，如何反映出传统心理学方法的局限性以及非暴力沟通的创新性？
Retrieved node with id, entering: 005ec9b0-fcaf-4ac7-abac-af136cffd927
Retrieving with query id 005ec9b0-fcaf-4ac7-abac-af136cffd927: 作者在致谢中提

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,small2big_retriever,0.11,0.105,0.027667,0.11,0.105,0.106309


In [175]:
simple_vector_eval_results = await simple_vector_retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)
display_results("simple_vector_retriever", simple_vector_eval_results)


100%|██████████| 100/100 [00:02<00:00, 34.48it/s]


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,simple_vector_retriever,0.59,0.348151,0.066619,0.59,0.348151,0.406403


MRR如何实现的？这里的mmr为什么总是接近hit_rate的一半数值？还是没有理解mrr的原理

### 检查dataset,自建测试集
- 构建测试集需要筛选出适合的节点用于构建，从字数上简单过滤先

In [ ]:
for query_id, _ in eval_dataset_zh.queries.items():
    expected_ids = eval_dataset_zh.relevant_docs[query_id]
    print(len(expected_ids))

In [204]:
QA_GENERATE_EASY = """\
You are an assistant helping users read Chinese books.
Based on the context below, generate {num_questions_per_chunk} **easy** questions that test basic understanding or facts.

Context:
---------------------
{context_str}
---------------------

Rules:
- Questions must be written in **Chinese**.
- Do not include answers or question numbers.
- Each question must be concise (<= 20 characters).
- Only ask about **explicit content** from the context (e.g., names, definitions, facts).
- Avoid "这", "那", "根据" and similar vague words.

Example question:
某理论的核心概念是?
"""

In [230]:
import random

# 筛选出hybrid_nodes中text长度>800的节点
print(len(hybrid_nodes))
hybrid_nodes_filtered = [node for node in hybrid_nodes if len(node.text) >= 800]
print("过滤后节点数", len(hybrid_nodes_filtered))

170
过滤后节点数 119


In [224]:
# 从中随机选择一个用于测试
test_node = random.sample(hybrid_nodes_filtered, 1)[0]
print(test_node.text)
print(len(test_node.text))

在受到指责时，很少有人还能把注意力放在对方的需要上。当然，也许这样的评判可以成功地胁迫他人满足我们的需要，他人会出于害怕、内疚或羞愧而改变行为，我们会就此以为指责他人能让自己“取得胜利”。 然而，以更长远的眼光来看，我们就会发现，若每一次都以这样的方式来满足需要，我们不仅无法胜利，还会助长地球上的暴力。为了解决眼前的问题，我们制造了新的问题。指责和评判使人倾向于自我保护并变得更有攻击性，日后也不可能关心我们的需要。因此，即便他人照着我们的意思满足了我们眼下的需要，之后我们也会为此付出代价。 表达愤怒的四个步骤 1. 停下来，深呼吸。 2. 看看我们有哪些评判性的想法。 3. 与我们的需要连结。 4. 表达我们的感受和未被满足的需要。 来看看充分表达愤怒的具体步骤。首先，停下来，除了深呼吸，什么都别做；不去指责或惩罚他人，只是保持安静。接着，看一看是什么想法让我们生气。例如，无意中听到某人说了一些话，使我们相信自己由于种族的缘故遭到了排斥。感知到自己的怒气时，如果停下来，便会看到头脑中盘旋的想法：“这太不公平了！她这是种族歧视！”我们知道，所有的评判都在用可悲的方式表达未满足的需要。于是，接下来的第3步是：与这些想法背后的需要连结。如果指责某人是种族主义者，我们的需要也许是融合、平等、尊重或者与人连结。 为了充分表达自己，我们要采取的第4步是开口表达愤怒——怒火此时已被转化为感受和需要。然而，表达此时的感受也许要很大的勇气。对我来说，生气并冲着人们嚷“你这样做是种族歧视”是很容易的，甚至还会感到很畅快。然而，要说出内心深处的感受和需要，却可能令人却步。为了充分表达愤怒，我们也许可以试着和对方说：“你走进房间后，开始与其他人聊天，却没有和我说话。接着，你开始议论有关白人的种种。当注意到这些时，我的胃非常不舒服，我也很害怕。这使我意识到，我非常希望受到平等的对待。我想请你告诉我，听到这些话，你的感受是怎么样的？” 先同理他人 在大多数情况下，如果希望他人听见我们的心声，与我们的内心产生连结，我们需要先同理对方。如果对方还处于某种情绪中，他们就很难平静下来体会我们的感受和需要。
900


In [226]:
text = test_node.text
query = QA_GENERATE_EASY.format(context_str=text, num_questions_per_chunk=2)
print(query)

You are an assistant helping users read Chinese books.
Based on the context below, generate 2 **easy** questions that test basic understanding or facts.

Context:
---------------------
在受到指责时，很少有人还能把注意力放在对方的需要上。当然，也许这样的评判可以成功地胁迫他人满足我们的需要，他人会出于害怕、内疚或羞愧而改变行为，我们会就此以为指责他人能让自己“取得胜利”。 然而，以更长远的眼光来看，我们就会发现，若每一次都以这样的方式来满足需要，我们不仅无法胜利，还会助长地球上的暴力。为了解决眼前的问题，我们制造了新的问题。指责和评判使人倾向于自我保护并变得更有攻击性，日后也不可能关心我们的需要。因此，即便他人照着我们的意思满足了我们眼下的需要，之后我们也会为此付出代价。 表达愤怒的四个步骤 1. 停下来，深呼吸。 2. 看看我们有哪些评判性的想法。 3. 与我们的需要连结。 4. 表达我们的感受和未被满足的需要。 来看看充分表达愤怒的具体步骤。首先，停下来，除了深呼吸，什么都别做；不去指责或惩罚他人，只是保持安静。接着，看一看是什么想法让我们生气。例如，无意中听到某人说了一些话，使我们相信自己由于种族的缘故遭到了排斥。感知到自己的怒气时，如果停下来，便会看到头脑中盘旋的想法：“这太不公平了！她这是种族歧视！”我们知道，所有的评判都在用可悲的方式表达未满足的需要。于是，接下来的第3步是：与这些想法背后的需要连结。如果指责某人是种族主义者，我们的需要也许是融合、平等、尊重或者与人连结。 为了充分表达自己，我们要采取的第4步是开口表达愤怒——怒火此时已被转化为感受和需要。然而，表达此时的感受也许要很大的勇气。对我来说，生气并冲着人们嚷“你这样做是种族歧视”是很容易的，甚至还会感到很畅快。然而，要说出内心深处的感受和需要，却可能令人却步。为了充分表达愤怒，我们也许可以试着和对方说：“你走进房间后，开始与其他人聊天，却没有和我说话。接着，你开始议论有关白人的种种。当注意到这些时，我的胃非常不舒服，我也很害怕。这使我意识到，我非常希望受到平等的对待。我想请你告诉我，听到这些话，你的感受是怎么样的

In [227]:
response = llm.complete(query)

In [228]:
print(response)

表达愤怒的第一步是什么？  
指责他人会带来什么后果？


In [234]:
sample_size = 50
hybrid_nodes_filtered_sample = random.sample(hybrid_nodes_filtered, sample_size)

In [241]:
# 基于hybrid_nodes_filtered构建语料数据集,并保存到csv文件中,包含三列，一行为node_id，一行为content，一行为questions
from tqdm import tqdm

rows = []
for node in tqdm(hybrid_nodes_filtered_sample, desc="构建语料数据集"):
    node_id = node.node_id
    content = node.text
    # 用content构建query，然后调用模型生成问题
    query = QA_GENERATE_EASY.format(context_str=content, num_questions_per_chunk=2)
    response = llm.complete(query)
    questions = str(response).strip().split("\n")
    questions = "#".join(questions)
    rows.append((node_id, content, questions))

# 将node_contents保存到csv文件中
df = pd.DataFrame(rows, columns=["node_id", "content", "questions"])
df.to_csv("qa_dataset.csv", index=False)


构建语料数据集: 100%|██████████| 50/50 [03:41<00:00,  4.44s/it]


In [236]:
# 加载数据集csv文件，转换为json
df = pd.read_csv("qa_dataset.csv")
df.columns

Index(['node_id', 'content', 'questions'], dtype='object')

In [ ]:
## 准备构建json数据集
test_qa_datasets = {}
relevant_docs = {}

In [242]:
# 根据df把各行的node_id,content按照node_id作为key，content作为value，构建corpus字典
corpus = {}
for index, row in df.iterrows():
    node_id = row["node_id"]
    content = row["content"]
    corpus[node_id] = content
print(len(corpus))
# 查看第一条数据

50


In [243]:
# 生成queries字典
# query_id通过str(uuid.uuid4())生成，作为key
# 列questions由于通过#连接的，需要分割成为两个问题，因此读取df每行的时候，会创建两个key：value对
# 同时更新relevant_docs字典，把query_id作为key,node_id作为value
import uuid

queries = {}
relevant_docs = {}
for index, row in df.iterrows():
    # 两个不同的query_id
    query_id_q1 = str(uuid.uuid4())
    query_id_q2 = str(uuid.uuid4())
    questions = row["questions"].replace("##", "#").split("#")
    questions = [question.strip() for question in questions]  # 去除前后空格
    queries[query_id_q1] = questions[0]  # 问题1
    relevant_docs[query_id_q1] = [row["node_id"]]
    queries[query_id_q2] = questions[1]  # 问题2
    relevant_docs[query_id_q2] = [row["node_id"]]

test_qa_datasets_json = {
    "queries": queries,
    "corpus": corpus,
    "relevant_docs": relevant_docs,
}
with open("test_qa_datasets.json", "w") as f:
    json.dump(test_qa_datasets_json, f, indent=4, ensure_ascii=False)


# 处理完记得检查数据集，可能还要进一步清洗数据

In [244]:
# 生成EmbeddingQAFinetuneDataset数据
eval_dataset_from_hybrid_nodes_50 = EmbeddingQAFinetuneDataset.from_json(
    "test_qa_datasets.json",
)


In [246]:
# 进行测试
eval_results_hybrid_nodes_50 = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_from_hybrid_nodes_50, show_progress=True
)


  0%|          | 0/100 [00:00<?, ?it/s]

Retrieving with query id None: 丈夫最担心的事情是什么？


2025-04-09 00:30:00.441 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:00.498 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:00.537 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  1%|          | 1/100 [00:00<01:26,  1.14it/s]2025-04-09 00:30:00.579 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:00.623 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: 1d9b4cb4-fce3-40c1-8e5b-bd457d1ecc60
Retrieving with query id 1d9b4cb4-fce3-40c1-8e5b-bd457d1ecc60: 丈夫最担心的事情是什么？
Retrieved node with id, entering: f579121b-f157-4233-b2f2-8201c024d73c
Retrieving with query id f579121b-f157-4233-b2f2-8201c024d73c: 丈夫最担心的事情是什么？
Retrieved node with id, entering: 94807d05-0b0c-4165-8b54-175bc2ea93fa
Retrieving with query id 94807d05-0b0c-4165-8b54-175bc2ea93fa: 丈夫最担心的事情是什么？
Retrieved node with id, entering: 7e01fa91-617f-4652-b129-db38a5bcf3e2
Retrieving with query id 7e01fa91-617f-4652-b129-db38a5bcf3e2: 丈夫最担心的事情是什么？
Retrieved node with id, entering: 69ae5f8e-3282-4aff-ab27-c70f849acead
Retrieving with query id 69ae5f8e-3282-4aff-ab27-c70f849acead: 丈夫最担心的事情是什么？
Retrieving with query id None: 病人被诊断为什么病？
Retrieved node with id, entering: fc5fb51d-24f4-451e-93c2-bd8c9863a608
Retrieving with query id fc5fb51d-24f4-451e-93c2-bd8c9863a608: 病人被诊断为什么病？
Retrieved node with id, entering: 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e
Retriev

2025-04-09 00:30:00.667 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:00.715 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  5%|▌         | 5/100 [00:01<00:16,  5.93it/s]2025-04-09 00:30:00.805 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:00.855 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  7%|▋         | 7/100 [00:01<00:12,  7.62it/s]

...

Retrieved node with id, entering: 794e0320-1392-4339-a80c-009435c45f6d
Retrieving with query id 794e0320-1392-4339-a80c-009435c45f6d: 作者如何应对愤怒情绪？
Retrieved node with id, entering: cde40d62-744a-4151-837c-83b2815d32fe
Retrieving with query id cde40d62-744a-4151-837c-83b2815d32fe: 作者如何应对愤怒情绪？
Retrieved node with id, entering: 6d70940f-2516-4102-a5b4-fca469925da8
Retrieving with query id 6d70940f-2516-4102-a5b4-fca469925da8: 作者如何应对愤怒情绪？
Retrieved node with id, entering: 5e0aabca-dff2-4bb6-b210-c6c4e232a086
Retrieving with query id 5e0aabca-dff2-4bb6-b210-c6c4e232a086: 作者如何应对愤怒情绪？
Retrieved node with id, entering: a8f63fdd-3893-4761-be42-114957027a8b
Retrieving with query id a8f63fdd-3893-4761-be42-114957027a8b: 作者如何应对愤怒情绪？
Retrieving with query id None: 爱比克泰德是哪国人？
Retrieved node with id, entering: 991577d6-5459-48cb-a0ba-fe3e7e7d0214
Retrieving with query id 991577d6-5459-48cb-a0ba-fe3e7e7d0214: 爱比克泰德是哪国人？
Retrieved node with id, entering: 5f44342d-77d1-4243-a602-6b0ecbfe5edf
Retrieving w

2025-04-09 00:30:03.909 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
 89%|████████▉ | 89/100 [00:04<00:00, 27.27it/s]2025-04-09 00:30:03.971 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.008 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.045 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.081 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
 93%|█████████▎| 93/100 [00:04<00:00, 26.17it/s]

Retrieved node with id, entering: d1c08388-f9be-4e47-845f-deed0afc0bd3
Retrieving with query id d1c08388-f9be-4e47-845f-deed0afc0bd3: 打断对话的最佳时机是？
Retrieved node with id, entering: ccafb2d8-78ab-4f16-8f0e-5f368871b850
Retrieving with query id ccafb2d8-78ab-4f16-8f0e-5f368871b850: 打断对话的最佳时机是？
Retrieved node with id, entering: c2c66937-abb3-4c45-a1e0-38a994926174
Retrieving with query id c2c66937-abb3-4c45-a1e0-38a994926174: 打断对话的最佳时机是？
Retrieving with query id None: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: a8f63fdd-3893-4761-be42-114957027a8b
Retrieving with query id a8f63fdd-3893-4761-be42-114957027a8b: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: 5508cb82-2abe-44f0-b9f1-72c5202c14d2
Retrieving with query id 5508cb82-2abe-44f0-b9f1-72c5202c14d2: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: 794e0320-1392-4339-a80c-009435c45f6d
Retrieving with query id 794e0320-1392-4339-a80c-009435c45f6d: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: e327daa0-0785-4d76-a805-8b512ef6dcac
Retrie

2025-04-09 00:30:04.116 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.150 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.186 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 00:30:04.221 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
100%|██████████| 100/100 [00:04<00:00, 21.76it/s]

Retrieved node with id, entering: 131094ea-58d6-4e8b-9010-29dd7d0f2e97
Retrieving with query id 131094ea-58d6-4e8b-9010-29dd7d0f2e97: 非暴力沟通的第二要素是？
Retrieved node with id, entering: 268fc4c6-0d99-4363-b503-75330fc2d7e0
Retrieving with query id 268fc4c6-0d99-4363-b503-75330fc2d7e0: 非暴力沟通的第二要素是？
Retrieved node with id, entering: ee702fc3-685b-442b-9f5a-84fe20693636
Retrieving with query id ee702fc3-685b-442b-9f5a-84fe20693636: 非暴力沟通的第二要素是？
Retrieved node with id, entering: 5e8705e5-6eee-42e9-8551-dae3d2843602
Retrieving with query id 5e8705e5-6eee-42e9-8551-dae3d2843602: 非暴力沟通的第二要素是？
Retrieved node with id, entering: 024a9f09-05ec-4745-a582-bb74c1b3754f
Retrieving with query id 024a9f09-05ec-4745-a582-bb74c1b3754f: 非暴力沟通的第二要素是？
Retrieved node with id, entering: 198eaf12-aab4-43ba-b95c-f9dd4579adb9
Retrieving with query id 198eaf12-aab4-43ba-b95c-f9dd4579adb9: 非暴力沟通的第二要素是？
Retrieved node with id, entering: 7b5a47cb-9add-42fd-a70f-b497ebbeefcf
Retrieving with query id 7b5a47cb-9add-42fd-a70

In [247]:
# 2025-04-08下午最新一次测试
display_results("hybrid_nodes_50", eval_results_hybrid_nodes_50)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.86,0.284075,0.111933,0.86,0.284075,0.424345


结果显示命中率不错，但mrr还是低，要从原理上分析其原因

### 自定义实现Retriever

In [129]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever, RecursiveRetriever


class HybridRetriever(BaseRetriever):
    def __init__(
        self,
        dense_retriever: RecursiveRetriever,
        sparse_retriever: BM25Retriever,
        retrieval_type=3,
        topk=256,
    ):
        self.dense_retriever = dense_retriever
        self.sparse_retriever = sparse_retriever
        self.retrieval_type = retrieval_type  # 1:dense only 2:sparse only 3:hybrid
        self.filters = None
        self.filter_dict = None
        self.topk = topk
        super().__init__()

    @classmethod
    def fusion(self, list_of_list_ranks_system, topk=256):
        all_nodes = []

        node_ids = set()
        for nodes in list_of_list_ranks_system:
            for node in nodes:
                content = node.get_content()
                if content not in node_ids:
                    all_nodes.append(node)
                    node_ids.add(content)
        all_nodes = sorted(all_nodes, key=lambda node: node.score, reverse=True)
        topk = min(len(all_nodes), topk)
        # print("simple fusion后数量:", topk)
        return all_nodes[:topk]

    # 倒数排序融合
    @classmethod
    def reciprocal_rank_fusion(self, list_of_list_ranks_system, K=60, topk=256):
        from collections import defaultdict

        rrf_map = defaultdict(float)
        text_to_node = {}
        for rank_list in list_of_list_ranks_system:
            for rank, item in enumerate(rank_list, 1):
                content = item.get_content()
                text_to_node[content] = item
                rrf_map[content] += 1 / (rank + K)
        sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

        reranked_nodes: list[NodeWithScore] = []
        for text, score in sorted_items:
            reranked_nodes.append(text_to_node[text])
            reranked_nodes[-1].score = score
        topk = min(topk, len(reranked_nodes))
        # print("rrf fusion后数量:", topk)
        return reranked_nodes[:topk]

    async def _aretrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        if self.retrieval_type != 1:
            self.sparse_retriever.filter_dict = self.filter_dict
            sparse_nodes = await self.sparse_retriever.aretrieve(query_bundle)
            if self.retrieval_type == 2:
                return sparse_nodes
        if self.retrieval_type != 2:
            self.dense_retriever.filters = self.filters
            dense_nodes = await self.dense_retriever.aretrieve(query_bundle)
            if self.retrieval_type == 1:
                return dense_nodes

        # combine the two lists of nodes
        # all_nodes = self.fusion(sparse_nodes, dense_nodes)
        all_nodes = self.reciprocal_rank_fusion(
            [sparse_nodes, dense_nodes], topk=self.topk
        )
        return all_nodes

    def _retrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        sparse_nodes = self.sparse_retriever.retrieve(query_bundle)
        dense_nodes = self.dense_retriever.retrieve(query_bundle)

        # combine the two lists of nodes
        all_nodes = self.reciprocal_rank_fusion(
            [sparse_nodes, dense_nodes], topk=self.topk
        )
        return all_nodes

### Hybrid test

In [ ]:
def reciprocal_rank_fusion(list_of_list_ranks_system, K=60, topk=256):
    from collections import defaultdict

    rrf_map = defaultdict(float)
    text_to_node = {}
    for rank_list in list_of_list_ranks_system:
        for rank, item in enumerate(rank_list, 1):
            content = item.get_content()
            text_to_node[content] = item
            rrf_map[content] += 1 / (rank + K)
    sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

    reranked_nodes: list[NodeWithScore] = []
    for text, score in sorted_items:
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score
    topk = min(topk, len(reranked_nodes))
    print("rrf fusion后数量:", topk)
    return reranked_nodes[:topk]

In [ ]:
final_results = reciprocal_rank_fusion(
    list_of_list_ranks_system=[sparse_nodes, dense_nodes]
)
for n in final_results:
    print(n.score, "\n", n.text, "\n--------\n")